# Introduzione

L'environment **anaconda** da utilizzare è **ddmcs**.  

# Importazione pacchetti

Importare i vari pacchetti necessari  per eseguire le azioni elencate nell'introduzione:
   - selenium (per navigare sulle pagine web)
   - request (scaricare dati da internet)
   - bs4 (beautiful soup, manipolare gli html)
   - os (navigare nei file del computer)
   - pandas (manipolazione/esportazione dataframes)
   - time (per i time sleep)
   - datetime (gestire i timestamp, ovvero le stringhe contenti date e orari)
   - tqdm (per le barre di progresso) 
   - re (lavorare con regex e le regular expressions)

In [1]:
#%% Pacchetti
## Importazione pacchetti per selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service # Selezionare il chrome driver
# Altre funzionalità di selenium
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.common.action_chains import ActionChains
#from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
#from selenium.common.exceptions import TimeoutException
#from webdriver_manager.chrome import ChromeDriverManager
#-------##-------##-------##-------##-------##-------##-------##-------##-------##-------#
import requests # Importazione pacchetti per scaricare dati da internet
from bs4 import BeautifulSoup # Importazione pacchetti per gestire HTML
import os # Interagire con i file del pc 
from tqdm import tqdm as tqdm # Importazione pacchetti per barra di progresso
import re # Importazione pacchetti per utilizzare regex e le regular expression
import time # Importazione pacchetti per utilizzare dei time.sleep()
# import pandas as pd  # Importazione pacchetti per lavorare con i dataframe
# import warnings # Importazione pacchetti per gestire i warnings, ad esempio quelli di pandas
import random # Importazione pacchetti per ottenere numeri casuali, utile per utilizzare time.sleep() randomici (meno identificabili)
#from datetime import datetime # Importazione pacchetti per lavorare con timestamps, orari e date



# Importare le funzioni per lavorare su instagram con il selenium both
from funzioni_selenium_bot_agoda import *

# Percorsi file 
Siccome questo bot leggerà e scriverà file localmente, stabilire quali sono i percorsi file di questi elementi.   

In [2]:
# Path per il driver di selenium
path_driver="/Users/mattia/opt/chromedriver137"

# Directory principale dello scraper, ove vi sono gli script e gli html devono essere posizionati
path_main_directory = '/Users/mattia/Desktop/Università/Data Science in Python/14) Web Scraping/Scraping-from-Agoda'

Creare le directory ove mettere i file

In [3]:
# Andare alla directory principale specificata nella precedente linea di codice
os.chdir(path_main_directory)

# Se non c'è la directory html crearla. Viene inserita questa clausola per evitare che una directory esistente
# venga cancellata
if "html" not in os.listdir():
   os.mkdir("html")


# Impostare il selenium bot e accesso ad Instagram

In [4]:
# La classe Service di Selenium viene utilizzata per gestire e configurare il driver del browser
service = Service(executable_path=path_driver)

# Configurare il driver di Chrome 
options = webdriver.ChromeOptions()


#options.add_argument("--headless")  # Modalità headless

# No images
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)


# Aggiungere l'utilizzo di proxy
#options.add_argument(f'--proxy-server={proxy_address}') # non sembra funzionare con la proxy, troppo lento

# Definire il driver: utilizzare il driver scelto (service) e passargli le impostazioni scelte (options)
driver = webdriver.Chrome(service=service, options=options)



# Impostare le dimensioni della finestra google sulla quale il bot selenium opera.
driver.maximize_window() # dimensioni massime
#driver.set_window_size(600, 800)   # dimensioni customizzate

# Impostare la posizione della finestra google sulla quale il bot selenium opera.
#driver.set_window_position(0,0)




# Istruzioni per il selenium bot 

In [5]:
link = """https://www.agoda.com/it-it/search?city=626&locale=it-it&ckuid=45687004-1d36-4d5b-a134-176448822573&prid=0&currency=EUR&correlationId=537c1df2-2a6a-42d7-8d28-855146282b2a&analyticsSessionId=7120213444848295795&pageTypeId=103&realLanguageId=4&languageId=4&origin=IT&stateCode=RM&cid=-1&userId=45687004-1d36-4d5b-a134-176448822573&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=1&currencyCode=EUR&htmlLanguage=it-it&cultureInfoName=it-it&machineName=am-pc-4g-acm-web-user-695fd75f86-nn987&trafficGroupId=4&trafficSubGroupId=4&aid=130243&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&browserFamily=Safari&cdnDomain=agoda.net&checkIn=2025-08-01&checkOut=2025-08-02&rooms=1&adults=2&children=0&priceCur=EUR&los=1&textToSearch=Terni&productType=-1&travellerType=1&familyMode=off&ds=A0fnajy9b9Jt5SPD"""
   

driver.get(link) 

time.sleep(10)


click_cookie_button(driver)


scroll_step = 200
last_height = driver.execute_script("return document.body.scrollHeight")
current_position = 0
scroll = 0

while True:
   # Prendere l'html della pagina 
   html=driver.page_source 
   


   html_bs4 = BeautifulSoup(html,'html.parser')
   # Identificare numero di pagine e pagine corrente  
   pagine = html_bs4.find("div",{"id":"paginationContainer"}).find("span",{"id":"paginationPageCount"}).text.replace("Pagina","")
   pagina_corrente = int(pagine.split("di")[0])
   pagine_totali = int(pagine.split("di")[1])
   #print(pagina_corrente,pagine_totali)
   
   # Salvare l'html in un file txt
   with open(f"{path_main_directory}/HTML/html_{scroll}.txt","w+") as f:
      f.write(str(html))    
      
    
   driver.execute_script(f"window.scrollBy(0, {scroll_step});")
   
   # Tempo di attesa randomico, meno riconoscibile
   tempo_attesa = random.randint(3,6)
   time.sleep(tempo_attesa)
   
   current_position += scroll_step
   last_height = driver.execute_script("return document.body.scrollHeight")
   scroll += 1
   


   # Se si è arrivati al finale della pagina e si è alla pagina finale interrompere
   if (current_position >= last_height) and (pagina_corrente == pagine_totali) :
      with open(f"{path_main_directory}/HTML/html_final_pagina{pagina_corrente}.txt","w+") as f:
         f.write(str(html))  
      break 
   

   # se si è arrivati al finale della pagina ma ci sono ancora pagine andare alla pagina successivaa
   elif (current_position >= last_height) and (pagina_corrente < pagine_totali) :
      # Salvare l'html in un file txt, quando si clicca si andrà su una nuova pagina, salvare tutti i risultati
      with open(f"{path_main_directory}/HTML/html_final_pagina{pagina_corrente}.txt","w+") as f:
         f.write(str(html))  

      current_position = 0
      click_avanti_button(driver)
      time.sleep(20)
      
      print("pagina successiva, si va a pagina:",pagina_corrente+1)
      
      continue 

      


   
driver.quit()

Cookie accettati metodo 1
Avanti cliccato metodo 1
pagina successiva, si va a pagina: 2
